<a href="https://colab.research.google.com/github/Ntorres95/BellyButtonBiodiversity/blob/master/DataCleaning_and_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3
!pip install fsspec
!pip install s3fs

     |████████████████████████████████| 133kB 7.0MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 6.9MB 16.1MB/s 
     |████████████████████████████████| 143kB 44.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 6.5MB 10.2MB/s 
     |████████████████████████████████| 1.3MB 36.9MB/s 
     |████████████████████████████████| 552kB 50.7MB/s 
     |████████████████████████████████| 133kB 50.7MB/s 
     |████████████████████████████████| 143kB 52.3MB/s 
     |█████████████████

In [2]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import boto3
import botocore
import os

In [3]:
from boto3.session import Session
from botocore.client import Config
from botocore.handlers import set_list_objects_encoding_type_url

In [4]:
weeks_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/weeksNFL.csv")
games_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/games.csv")
players_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/players.csv")
plays_df = pd.read_csv("https://mybigbowlbucket.s3-us-west-1.amazonaws.com/plays.csv")

In [5]:
weeks_df.shape

(18309388, 19)

In [6]:
weeks_df = weeks_df.dropna()
games_df = games_df.dropna()
players_df = players_df.dropna()
plays_df = plays_df.dropna()

In [7]:
weeks_df.shape

(5117955, 19)

In [8]:
weeks_df.head(3)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH
10,2018-09-07T01:07:14.599Z,91.29,21.83,0.01,0.02,0.01,250.34,151.83,None,2555415.0,Austin Hooper,81.0,TE,1,away,2018090600,75,left,OUT


In [9]:
games_df.head(3)

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,09/06/2018,20:20:00,PHI,ATL,1
1,2018090901,09/09/2018,13:00:00,CLE,PIT,1
2,2018090902,09/09/2018,13:00:00,IND,CIN,1


In [10]:
players_df.head(3)

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen


In [11]:
plays_df.head(3)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
29,2018090600,1680,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.299050,False
45,2018090600,2503,(11:04) (Shotgun) M.Ryan pass incomplete short...,3,2,10,ATL,play_type_pass,ATL,32,EMPTY,"1 RB, 1 TE, 3 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,6.0,3.0,11:04:00,42.0,ING,ATL 2,I,0,-12,-1.868122,False
91,2018090901,439,(8:05) (Shotgun) B.Roethlisberger pass short r...,1,3,9,PIT,play_type_pass,PIT,43,EMPTY,"1 RB, 1 TE, 3 WR",6.0,4.0,"2 DL, 4 LB, 5 DB",TRADITIONAL,0.0,0.0,08:05:00,53.0,IBW,PIT 11,C,0,0,-1.770704,False


In [12]:
df = weeks_df.merge(plays_df, how='right', on='playId')
df.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId_x,playId,playDirection,route,gameId_y,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018-09-07T02:17:33.400Z,110.35,32.48,0.00,0.00,0.00,44.48,307.08,None,2506467.0,Darren Sproles,43.0,RB,1.0,home,2.018091e+09,1680,left,WHEEL,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
1,2018-09-07T02:17:33.400Z,106.49,42.95,0.00,0.00,0.00,247.94,189.89,None,2507763.0,Mike Wallace,14.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
2,2018-09-07T02:17:33.400Z,105.99,22.35,0.00,0.00,0.00,255.39,246.65,None,2540158.0,Zach Ertz,86.0,TE,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
3,2018-09-07T02:17:33.400Z,107.67,16.05,5.83,2.36,0.59,327.16,2.02,None,2552600.0,Nelson Agholor,13.0,WR,1.0,home,2.018091e+09,1680,left,CROSS,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False
4,2018-09-07T02:17:33.400Z,106.75,15.20,0.00,0.00,0.00,253.47,288.27,None,2553502.0,DeAndre Carter,16.0,WR,1.0,home,2.018091e+09,1680,left,POST,2018090600,(1:57) (Shotgun) N.Foles pass short right to N...,2,2,10,PHI,play_type_pass,PHI,5,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,6.0,3.0,01:57:00,105.0,RPS,ATL 97,C,4,19,1.29905,False


In [13]:
df = df.drop(columns=['time', 'x', 'y', 's', 'a', 'o', 'dir', 'event', 'nflId', 'displayName', 'jerseyNumber', 'frameId', 'gameId_x', 'gameId_y', 'playDescription','possessionTeam', 'playType', 'yardlineSide', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers','typeDropback'])

In [14]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

df = pd.get_dummies(df)

df.head()

,dis,playId,quarter,down,yardsToGo,yardlineNumber,defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore,absoluteYardlineNumber,offensePlayResult,playResult,epa,isDefensivePI,position_FB,position_HB,position_QB,position_RB,position_TE,position_WR,team_away,team_home,playDirection_left,playDirection_right,route_ANGLE,route_CORNER,route_CROSS,route_FLAT,route_GO,route_HITCH,route_IN,route_OUT,route_POST,route_SCREEN,route_SLANT,route_WHEEL,route_undefined,offenseFormation_EMPTY,offenseFormation_I_FORM,...,offenseFormation_WILDCAT,"personnelO_0 RB, 0 TE, 5 WR","personnelO_0 RB, 1 TE, 4 WR","personnelO_1 RB, 0 TE, 4 WR","personnelO_1 RB, 1 TE, 3 WR","personnelO_1 RB, 2 TE, 2 WR","personnelO_1 RB, 3 TE, 1 WR","personnelO_2 QB, 1 RB, 1 TE, 2 WR","personnelO_2 RB, 0 TE, 3 WR","personnelO_2 RB, 1 TE, 2 WR","personnelO_2 RB, 2 TE, 1 WR","personnelO_3 RB, 1 TE, 1 WR","personnelO_6 OL, 1 RB, 1 TE, 2 WR","personnelO_6 OL, 1 RB, 2 TE, 1 WR","personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL","personnelO_6 OL, 2 RB, 2 TE, 0 WR","personnelD_0 DL, 4 LB, 7 DB","personnelD_0 DL, 5 LB, 6 DB","personnelD_0 DL, 6 LB, 5 DB","personnelD_1 DL, 3 LB, 7 DB","personnelD_1 DL, 4 LB, 6 DB","personnelD_1 DL, 5 LB, 5 DB","personnelD_2 DL, 2 LB, 7 DB","personnelD_2 DL, 3 LB, 6 DB","personnelD_2 DL, 4 LB, 5 DB","personnelD_3 DL, 1 LB, 7 DB","personnelD_3 DL, 2 LB, 6 DB","personnelD_3 DL, 3 LB, 5 DB","personnelD_3 DL, 4 LB, 4 DB","personnelD_3 DL, 5 LB, 3 DB","personnelD_4 DL, 1 LB, 6 DB","personnelD_4 DL, 2 LB, 5 DB","personnelD_4 DL, 3 LB, 4 DB","personnelD_4 DL, 4 LB, 3 DB","personnelD_4 DL, 5 LB, 2 DB","personnelD_5 DL, 2 LB, 4 DB",passResult_C,passResult_I,passResult_IN,passResult_S
0,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0.59,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0.00,1680,2,2,10,5,6.0,4.0,6.0,3.0,105.0,4,19,1.29905,False,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [15]:
# Set features. This will also be used as your x values.
selected_features = df[['quarter', 'down', 'yardsToGo', 'defendersInTheBox', 'numberOfPassRushers', 'isDefensivePI',
                        'offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',
                        'personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB'	]]

In [16]:
defensive_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	'personnelD_1 DL, 4 LB, 6 DB',
                        'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB',	'personnelD_2 DL, 3 LB, 6 DB',	'personnelD_2 DL, 4 LB, 5 DB',	'personnelD_3 DL, 1 LB, 7 DB',
                        'personnelD_3 DL, 2 LB, 6 DB',	'personnelD_3 DL, 3 LB, 5 DB',	'personnelD_3 DL, 4 LB, 4 DB',	'personnelD_3 DL, 5 LB, 3 DB',	'personnelD_4 DL, 1 LB, 6 DB',
                        'personnelD_4 DL, 2 LB, 5 DB',	'personnelD_4 DL, 3 LB, 4 DB',	'personnelD_4 DL, 4 LB, 3 DB',	'personnelD_4 DL, 5 LB, 2 DB',	'personnelD_5 DL, 2 LB, 4 DB']]

In [17]:
offensive_features = df[['offenseFormation_EMPTY',	'offenseFormation_I_FORM',	'offenseFormation_JUMBO',	'offenseFormation_PISTOL',	'offenseFormation_SHOTGUN',	'offenseFormation_SINGLEBACK',
                        'offenseFormation_WILDCAT',	'personnelO_0 RB, 0 TE, 5 WR',	'personnelO_0 RB, 1 TE, 4 WR',	'personnelO_1 RB, 0 TE, 4 WR',	'personnelO_1 RB, 1 TE, 3 WR',	'personnelO_1 RB, 2 TE, 2 WR',	
                        'personnelO_1 RB, 3 TE, 1 WR',	'personnelO_2 QB, 1 RB, 1 TE, 2 WR',	'personnelO_2 RB, 0 TE, 3 WR',	'personnelO_2 RB, 1 TE, 2 WR',	'personnelO_2 RB, 2 TE, 1 WR',
                        'personnelO_3 RB, 1 TE, 1 WR',	'personnelO_6 OL, 1 RB, 1 TE, 2 WR',	'personnelO_6 OL, 1 RB, 2 TE, 1 WR',	'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL',	'personnelO_6 OL, 2 RB, 2 TE, 0 WR',]]

**Catch Predictions**

Logistic Regression-----

In [18]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [19]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7214973850812001
Testing Data Score: 0.7213546346984276


Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9779619276148657

In [23]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.22615246517729037, 'yardsToGo'),
 (0.13663000440013245, 'quarter'),
 (0.1150892575014145, 'defendersInTheBox'),
 (0.10139439288855412, 'numberOfPassRushers'),
 (0.08494255365100666, 'down'),
 (0.028002106789086913, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.02598287368285114, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.023935920961247158, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.02299116026770439, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.01957712067183225, 'offenseFormation_SHOTGUN'),
 (0.01882183799518611, 'isDefensivePI'),
 (0.0178848820561276, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.016397497637150493, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.014702791233921788, 'offenseFormation_EMPTY'),
 (0.014145465235527255, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.013681601015266932, 'offenseFormation_SINGLEBACK'),
 (0.013668029345777743, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.012806214209865918, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.010046316697151387, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.00847938067835452, 'personnelO_2 RB, 1 TE, 2 

Random Forest Classifier

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9779619276148657

In [25]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.22512762410986023, 'yardsToGo'),
 (0.13452840260748256, 'quarter'),
 (0.11534199390877775, 'defendersInTheBox'),
 (0.10148753843184954, 'numberOfPassRushers'),
 (0.08330177027827228, 'down'),
 (0.028517061108061337, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.026847035638184788, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.024211542129644322, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.022079417951404605, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.019283620234093315, 'isDefensivePI'),
 (0.01869322724640635, 'offenseFormation_SHOTGUN'),
 (0.01701650644077443, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.01686358388679135, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.016784818906355362, 'offenseFormation_EMPTY'),
 (0.015439388950832779, 'offenseFormation_SINGLEBACK'),
 (0.013653686850181613, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.013151081151878059, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.012837806737839283, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.010311036876460709, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.008183621981206764, 'personnelO_2 RB, 1 T

**Incomplete Predictions**

Logistic Regression

In [26]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [27]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7959731869626463
Testing Data Score: 0.7970573805927013


Random Forest Classifier

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9910040753675845

In [31]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.2422791739233051, 'yardsToGo'),
 (0.1481419868672976, 'quarter'),
 (0.11851659040975127, 'numberOfPassRushers'),
 (0.10476670891035254, 'defendersInTheBox'),
 (0.078425641830812, 'down'),
 (0.02730228540284769, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.025200925364656304, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.020820463273132347, 'offenseFormation_SHOTGUN'),
 (0.0198770517828714, 'offenseFormation_EMPTY'),
 (0.01924648304785656, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.018918790430514167, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.01635910071045887, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.016139315648186957, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.013031320663495128, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.01230240892654137, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.012119169235695077, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.011646480878420639, 'offenseFormation_SINGLEBACK'),
 (0.010564217726611348, 'isDefensivePI'),
 (0.010415056460980787, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.00991646103488696, 'personnelD_4 DL, 3 LB, 4 DB'

**Sack Predictions**

Logistic Regression

In [32]:
from sklearn.model_selection import train_test_split
y = df["passResult_S"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [33]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9300804714949563
Testing Data Score: 0.930009569099096


Random Forest Clasifier

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9924612985801788

In [37]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.24882610729397833, 'yardsToGo'),
 (0.143481743522934, 'quarter'),
 (0.12494598928985287, 'numberOfPassRushers'),
 (0.1137552985997179, 'defendersInTheBox'),
 (0.0882517824000621, 'down'),
 (0.029315625473239532, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.023337237459191057, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.02149665981265191, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.01912472841428102, 'offenseFormation_SHOTGUN'),
 (0.01774893860988035, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.01681278666372922, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.016429412874692503, 'offenseFormation_SINGLEBACK'),
 (0.016240815803192393, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.015810678069829016, 'offenseFormation_I_FORM'),
 (0.011516321429141647, 'offenseFormation_EMPTY'),
 (0.011101650410442712, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.010448907475137836, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.010080782146640065, 'offenseFormation_PISTOL'),
 (0.009287393582733639, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.008785008068271979, 'personnelD_4 DL, 1 

**Interceptions Predictions**

Logistic Regression

In [38]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [39]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [40]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9646637845889801
Testing Data Score: 0.9644826128983683


Random Forest Classifier

In [42]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9946568482204876

In [43]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.15037972207936548, 'quarter'),
 (0.14824162068442756, 'numberOfPassRushers'),
 (0.11629316236457214, 'yardsToGo'),
 (0.1098671591804076, 'defendersInTheBox'),
 (0.08932014170729179, 'down'),
 (0.043681712950619636, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.03097080652264208, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.026403974845923474, 'offenseFormation_SHOTGUN'),
 (0.023668492467245034, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.02219586620032882, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.021862565853977965, 'offenseFormation_EMPTY'),
 (0.021474011515312065, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.020649455165868934, 'offenseFormation_WILDCAT'),
 (0.020280846882044786, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.017456020493570677, 'offenseFormation_I_FORM'),
 (0.013617086270676171, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.013031965747524871, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.013031199967967316, 'personnelO_2 QB, 1 RB, 1 TE, 2 WR'),
 (0.01249586925495564, 'offenseFormation_PISTOL'),
 (0.011840539124466809, 'personnelO_1

**Cacth Predictions vs Defensive formations**

Logistic Regression

In [44]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = defensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [45]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [46]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6863975648062693
Testing Data Score: 0.6853466491152227


Random Forest Classifier

In [48]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6853466491152227

In [49]:
sorted(zip(rf.feature_importances_, defensive_features), reverse=True)

[(0.1842372627919541, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.16848704442226212, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.1521402112708025, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.06434848967506446, 'personnelD_2 DL, 3 LB, 6 DB'),
 (0.05541927578519531, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.053595884298919275, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.050032522560008307, 'personnelD_4 DL, 3 LB, 4 DB'),
 (0.03956257506221424, 'personnelD_4 DL, 5 LB, 2 DB'),
 (0.03716086304808634, 'personnelD_3 DL, 4 LB, 4 DB'),
 (0.036477313382731004, 'personnelD_3 DL, 3 LB, 5 DB'),
 (0.03100240526197016, 'personnelD_4 DL, 4 LB, 3 DB'),
 (0.02878628031815073, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.019840077196773498, 'personnelD_5 DL, 2 LB, 4 DB'),
 (0.01670151236285351, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.014219774976207076, 'personnelD_4 DL, 2 LB, 5 DB'),
 (0.013866538955064251, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.01166988655926465, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.010318712299695236, 'personnelD_3 DL, 5 LB, 3 DB'),
 (0.0

**Incomplete Passes vs Offense formations**

Logistic Regression

In [50]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = offensive_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [51]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [52]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7736824209452567
Testing Data Score: 0.7738292425839483


Random Forest Classifier

In [54]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7765931092771687

In [55]:
sorted(zip(rf.feature_importances_, offensive_features), reverse=True)

[(0.13031664216048855, 'offenseFormation_SHOTGUN'),
 (0.12177153321622355, 'offenseFormation_SINGLEBACK'),
 (0.10822883881679746, 'personnelO_1 RB, 1 TE, 3 WR'),
 (0.09956355837314497, 'personnelO_1 RB, 0 TE, 4 WR'),
 (0.09284004627483664, 'offenseFormation_EMPTY'),
 (0.09156780771580575, 'personnelO_2 RB, 0 TE, 3 WR'),
 (0.06445590667016114, 'personnelO_0 RB, 1 TE, 4 WR'),
 (0.04971842109271029, 'personnelO_1 RB, 2 TE, 2 WR'),
 (0.04113588574576891, 'offenseFormation_I_FORM'),
 (0.04108203843744499, 'personnelO_2 RB, 1 TE, 2 WR'),
 (0.04078446536736185, 'offenseFormation_PISTOL'),
 (0.026720392703385693, 'personnelO_6 OL, 2 RB, 2 TE, 0 WR'),
 (0.02548130045419717, 'personnelO_1 RB, 3 TE, 1 WR'),
 (0.014889647695617729, 'personnelO_2 QB, 1 RB, 1 TE, 2 WR'),
 (0.011035702040815313, 'personnelO_2 RB, 2 TE, 1 WR'),
 (0.009746872191214331, 'personnelO_3 RB, 1 TE, 1 WR'),
 (0.00852062925163732, 'personnelO_6 OL, 2 RB, 1 TE, 0 WR,1 DL'),
 (0.005749990023647565, 'offenseFormation_JUMBO'),
 (0

**Exta DB's and Incomplete Catches**

Logistic Regression

In [56]:
moreDB_features = df[['personnelD_0 DL, 4 LB, 7 DB',	'personnelD_0 DL, 5 LB, 6 DB',	'personnelD_0 DL, 6 LB, 5 DB',	'personnelD_1 DL, 3 LB, 7 DB',	
                      'personnelD_1 DL, 4 LB, 6 DB', 'personnelD_1 DL, 5 LB, 5 DB',	'personnelD_2 DL, 2 LB, 7 DB', 'personnelD_2 DL, 4 LB, 5 DB',	
                      'personnelD_3 DL, 1 LB, 7 DB', 'personnelD_3 DL, 2 LB, 6 DB', 'personnelD_4 DL, 1 LB, 6 DB']]

In [57]:
from sklearn.model_selection import train_test_split
y = df["passResult_I"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [58]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [59]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7872427583750263
Testing Data Score: 0.7872891276576108


Random Forest Classifier

In [61]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7872891276576108

In [62]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.3060838868595242, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.27887434483118756, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.24232789123788714, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.09935220178782751, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.044684821853001766, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0087929107318632, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.007367208040295456, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.00651122202300255, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.005432967431615581, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.0004724111596411282, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.0001001340441539179, 'personnelD_1 DL, 5 LB, 5 DB')]

**Extra DB and Interceptions**

Logistic Regression

In [63]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [64]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [89]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9533475818072894
Testing Data Score: 0.953349427554148


In [97]:
import joblib
filename = 'model_db_&_intercepts.sav'
joblib.dump(model, filename) 

['model_db_&_intercepts.sav']

Random Forest Classifier

In [67]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)



0.9620539075440445

In [68]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.9109520502668937, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.02821577839568794, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.014698367903668208, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.013543042258718654, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.012697428808308153, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.007964733898087592, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0038954029206606654, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.003331215225925742, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.00272014895853972, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.0011591787784307054, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.0008226525850789816, 'personnelD_0 DL, 4 LB, 7 DB')]

**Extra DB and Sacks**

Logistic Regression

In [69]:
from sklearn.model_selection import train_test_split
y = df["passResult_IN"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [70]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [71]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9621136315797996
Testing Data Score: 0.9620539075440445


Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)


0.9620539075440445

In [74]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.9103162809560156, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.02870673645199447, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.01493026843520829, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.013740527186688663, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.012817012726521668, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.007880834254393228, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0037926742746472054, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.003264111973844091, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.0026238203063475205, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.0011165237445912575, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.0008112096897479774, 'personnelD_0 DL, 4 LB, 7 DB')]

**Extra DB's and Catches**

Logistic Regression

In [75]:
from sklearn.model_selection import train_test_split
y = df["passResult_C"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [76]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [77]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6844157316105633
Testing Data Score: 0.6834814034031019


Random Forest Classifier

In [79]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6834814034031019

In [80]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.28337068963354195, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.2422292663240744, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.22078013149814602, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.07961204702113264, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.07810783948021344, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.042838506626281046, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.024563851897931664, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.013014978351779678, 'personnelD_0 DL, 4 LB, 7 DB'),
 (0.008444784342860308, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.006497820116478128, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.0005400847075607911, 'personnelD_3 DL, 1 LB, 7 DB')]

**Offensive Formation agaisnt Extra DB's**

Logistic Regression

In [81]:
from sklearn.model_selection import train_test_split
y = df["offenseFormation_I_FORM"]
X = moreDB_features

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [82]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [83]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [84]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9533475818072894
Testing Data Score: 0.953349427554148


In [ ]:
model.save

Random Forest

In [85]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.953349427554148

In [86]:
sorted(zip(rf.feature_importances_, moreDB_features), reverse=True)

[(0.28666204271302775, 'personnelD_3 DL, 2 LB, 6 DB'),
 (0.2821982281463631, 'personnelD_4 DL, 1 LB, 6 DB'),
 (0.21589446148231722, 'personnelD_2 DL, 4 LB, 5 DB'),
 (0.08915278009335122, 'personnelD_1 DL, 4 LB, 6 DB'),
 (0.02705106460832037, 'personnelD_1 DL, 5 LB, 5 DB'),
 (0.026540554303834465, 'personnelD_1 DL, 3 LB, 7 DB'),
 (0.02510719938473439, 'personnelD_2 DL, 2 LB, 7 DB'),
 (0.021004050114851867, 'personnelD_3 DL, 1 LB, 7 DB'),
 (0.009383768744838745, 'personnelD_0 DL, 6 LB, 5 DB'),
 (0.009363736087964071, 'personnelD_0 DL, 5 LB, 6 DB'),
 (0.0076421143203967, 'personnelD_0 DL, 4 LB, 7 DB')]